In [0]:
# ============================================================================
# 🔒 SETUP_SECRETS.ipynb - DATABRICKS SECRETS VERSION
# ============================================================================
print("🔐 CONFIGURING DATABRICKS SECRETS")
print("=" * 50)

# ----------------------------------------------------------------------------
# YOUR ACTUAL API KEY (Only place you need to put it)
# ----------------------------------------------------------------------------
YOUR_REAL_EIA_API_KEY = "RLbkxvHkqPRfBuOzlfMjYUqlngpw6VjYEfvMAYO8"  # ← REPLACE WITH YOUR KEY

print(f"📋 Your key length: {len(YOUR_REAL_EIA_API_KEY)} characters")

# ----------------------------------------------------------------------------
# STEP 1: CREATE SECRET SCOPE (One-time setup)
# ----------------------------------------------------------------------------
print("\n1. Creating secret scope...")
try:
    # Check if scope exists
    scopes = dbutils.secrets.listScopes()
    scope_exists = any(scope.name == "eia-scope" for scope in scopes)
    
    if not scope_exists:
        # Create the scope (needs admin rights or proper permissions)
        dbutils.secrets.createScope(name="eia-scope")
        print("   ✅ Created new scope: 'eia-scope'")
    else:
        print("   ✅ Scope 'eia-scope' already exists")
        
except Exception as e:
    print(f"   ⚠️  Scope creation failed (may lack permissions): {str(e)[:100]}")

# ----------------------------------------------------------------------------
# STEP 2: STORE KEY IN SECRETS
# ----------------------------------------------------------------------------
print("\n2. Storing API key in secrets...")
try:
    # Store the key
    dbutils.secrets.put(scope="eia-scope", key="eia_api_key", value=YOUR_REAL_EIA_API_KEY)
    print("   ✅ Key stored: secrets/eia-scope/eia_api_key")
    
    # Verify it was stored
    test_key = dbutils.secrets.get(scope="eia-scope", key="eia_api_key")
    if test_key == YOUR_REAL_EIA_API_KEY:
        print("   ✅ Verification: Key retrieved successfully")
    else:
        print("   ⚠️  Verification: Retrieved key doesn't match")
        
except Exception as e:
    error_msg = str(e)
    print(f"   ❌ Failed to store in secrets: {error_msg[:100]}")
    
    if "PERMISSION_DENIED" in error_msg:
        print("\n   🔧 FIX NEEDED: You need proper permissions to create secrets.")
        print("   For Community Edition, we'll use environment variable instead.")
        
        # Fallback for Community Edition
        import os
        os.environ['EIA_API_KEY'] = YOUR_REAL_EIA_API_KEY
        print("   ✅ Fallback: Stored in environment variable")

# ----------------------------------------------------------------------------
# STEP 3: CREATE CONVENIENCE FUNCTION
# ----------------------------------------------------------------------------
print("\n3. Creating loader function for main notebook...")

loader_code = '''
def load_eia_api_key():
    """
    Load EIA API key from Databricks Secrets with fallbacks.
    Call this function in your main notebook.
    """
    # Try Databricks Secrets first (most secure)
    try:
        api_key = dbutils.secrets.get(scope="eia-scope", key="eia_api_key")
        if api_key:
            masked = api_key[:5] + "..." + api_key[-5:] if len(api_key) > 10 else "***"
            print(f"✅ Loaded from Databricks Secrets: {masked}")
            return api_key
    except:
        pass  # Continue to fallback
    
    # Fallback: Environment variable
    import os
    api_key = os.environ.get("EIA_API_KEY")
    if api_key:
        masked = api_key[:5] + "..." + api_key[-5:] if len(api_key) > 10 else "***"
        print(f"✅ Loaded from environment variable: {masked}")
        return api_key
    
    # Last resort: Hardcoded (NOT recommended)
    print("⚠️  WARNING: Using hardcoded key (insecure!)")
    return "RLbkxvHkqPRfBuOzlfMjYUqlngpw6VjYEfvMAYO8"
'''

print("   ✅ Function 'load_eia_api_key()' is ready to use")
print("\n" + "=" * 50)
print("✅ SETUP COMPLETE!")
print("\n📋 How to use in your main notebook:")
print("   1. Copy the 'load_eia_api_key()' function")
print("   2. Call it: EIA_API_KEY = load_eia_api_key()")
print("   3. That's it - your key is secure!")